In [3]:
%%time 

import numpy as np
import pandas as pd
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

pd.set_option('display.max_columns', None)
comp = pd.read_csv('~/misp_data/comp_1959-2019.csv', index_col=0)
crsp = pd.read_csv('~/misp_data/crsp_1959-2019.csv', index_col=0)
beta = pd.read_csv('~/misp_data/capm_beta_weekly_1-3y_1959-2019.csv', index_col=0)

comp['jdate'] = pd.to_datetime(comp['jdate'])
comp.permno = comp.permno.astype(int)

beta['date']=pd.to_datetime(beta['DATE'].astype(str))
beta['jdate']=beta['date']+MonthEnd(0)
beta['permno'] = beta.index
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

CPU times: user 1min 3s, sys: 4.57 s, total: 1min 7s
Wall time: 1min 7s


In [4]:
FF = pd.read_csv('~/misp/lookup_tables/FF3.csv')
FF['date'] = FF['Unnamed: 0']
FF['date'] = pd.to_datetime(FF['date'].astype(str), format='%Y%m%d')
FF['jdate'] = FF['date']+MonthEnd(0)
FF = FF[['jdate', 'date', 'RF']]
FF['RF'] = (1+FF['RF']/100).pow(365)-1

beta = beta.merge(FF[['jdate', 'RF']],  on=['jdate'])
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

In [5]:
def perform_valuation(df, val_method='dcf', **kwargs):
    if val_method == 'dcf':
        df['PV'] = dcf_valuation(df, **kwargs)
    else:
        raise ValueError(
            f'Invalid valuation method {val_method}.')
    return df


def dcf_valuation(df, n=10, rp=0.06, tg=0.06):
    beta = df['b_mkt']
    rf = df['RF']
    r = rf + beta * rp  # CAPM discount rate
    num_firms = df.shape[0]
    PV = np.zeros(num_firms)
    for i in range(n):
        ni_in_period_i = df[f'ni_{i+1}']
        PV += ni_in_period_i / (1 + r) ** i
        if i == n:
            PV += ni_in_period_i * (1 + tg) / (((1 + r) ** n) * (r - tg + 1e-10))
    return PV

In [6]:
rf_val = pd.read_csv('RF_ni_pred_for_val.csv')
rf_val['jdate'] = pd.to_datetime(rf_val['jdate'])

In [16]:
%%time
valdf = rf_val.merge(beta, on=['jdate', 'permno'])
valdf = perform_valuation(valdf, n=10)
valdf = valdf[['jdate', 'permno', 
               'year', 'PV']]
valdf.year = valdf.year + 1
val_crsp_mdf = valdf.merge(crsp, on=['year', 'permno'])
val_crsp_mdf['vt'] = val_crsp_mdf['PV']*1000

CPU times: user 4.8 s, sys: 2.33 s, total: 7.13 s
Wall time: 7.14 s


In [17]:
val_df1 = val_crsp_mdf[['vt', 'permno', 'ticker', 'me', 'year', 'jdate_crsp',
                        'logret_3', 'logret_6', 'logret_24', 'cumret_3', 'cumret_6', 'cumret_24',
                        'logret_12', 'logret_36', 'logret_60', 'cumret_12', 'cumret_36', 'cumret_60',
                         'termlogret', 'termcumret']]
val_df1.sort_values(['permno','jdate_crsp'], ascending = [True,False], inplace=True)

/home/mma3/.conda/envs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [18]:
val_df2 = val_df1.copy()
for m in [3, 6, 12, 24, 36, 60]:
    val_df2[f'vt+{m}'] = val_df1.groupby('permno')['vt'].shift(m)
    val_df2[f'me+{m}'] = val_df1.groupby('permno')['me'].shift(m)

In [19]:
val_df3 = val_df2.copy()
for m in [3, 6, 12, 24, 36, 60]:
    val_df3['vt'] = np.log(val_df2.vt)
    val_df3[f'vt+{m}'] = np.log(val_df2[f'vt+{m}'].astype(float))
    val_df3['me'] = np.log(val_df2['me'].astype(float))
    val_df3[f'me+{m}'] = np.log(val_df2[f'me+{m}'].astype(float))
    val_df3[f'r_t+{m}'] = val_df3[f'logret_{m}'].fillna(val_df3['termlogret']+np.log(1-0.35))
    val_df3[f'r*_t+{m}'] = val_df3[f'vt+{m}'] -  val_df3[f'vt']
    val_df3[f'r_unchg+{m}'] = - (1-0.975) * (val_df3['me'] -  val_df3['vt'])
    val_df3[f'r_chg+{m}'] = 0.975 * ( (val_df3[f'me+{m}'] -  val_df3[f'vt+{m}']) - ((val_df3[f'me'] -  val_df3['vt'])) )
    

/home/mma3/.conda/envs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/home/mma3/.conda/envs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


In [20]:
val_df3 = val_df3[1970<=val_df3.year]
val_df3 = val_df3.drop_duplicates(subset=['permno', 'jdate_crsp'])

In [21]:
val_df3['misp'] = val_df3['me'] - val_df3['vt']
val_df3 = val_df3.dropna(subset=['misp', 'r_t+60'])

In [22]:
val_df3[val_df3.ticker=='TLSA']

,vt,permno,ticker,me,year,jdate_crsp,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,vt+6,me+6,vt+12,me+12,vt+24,me+24,vt+36,me+36,vt+60,me+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp


In [14]:
%%time
val_df3.to_csv('~/misp_data/decomp_dcf10y_RF_1970_2019.csv')

CPU times: user 2min 26s, sys: 800 ms, total: 2min 27s
Wall time: 2min 28s
